In [ ]:
!pip install datasets

from datasets import load_dataset

# Load TinyStories (the small version)
dataset = load_dataset("roneneldan/TinyStories")

# Show an example
print(dataset["train"][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00003-of-00004-d243063613e5a0(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/validation-00000-of-00001-869c898b5(…):   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}


In [ ]:
# Checking the paragraphs
for i in range(5):
    print("text", i+1)
    print(dataset["train"][i]["text"])
    print("-----------")

text 1
One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.
-----------
text 2
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.

One day, Beep was driving in the park when he saw a big tree. The tree had many l

In [ ]:
# Extract all texts from train split
texts = [item["text"] for item in dataset["train"]]

Keep the paragraphs between 300-350 words and ends with a full sentence

In [ ]:
import re

# Split text into clean sentences
def split_sentences(text):
    # Basic sentence split using punctuation
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    # Remove empty sentences
    return [s.strip() for s in sentences if len(s.strip()) > 0]

# Build paragraph of 300–360 words ending with complete sentence
def build_paragraph(start_index, min_words=300, max_words=350):
    sentences = []
    idx = start_index
    word_count = 0

    # PHASE 1 -> reach minimum words (may overshoot, but we'll fix)
    while word_count < min_words:
        story_sentences = split_sentences(texts[idx])
        sentences.extend(story_sentences)
        word_count = len(" ".join(sentences).split())
        idx += 1

        if idx >= len(texts):
            break

    # PHASE 2 -> cap at max_words, add sentence-by-sentence only
    final_sentences = []
    current_words = 0

    for sent in sentences:
        sent_words = len(sent.split())

        # If adding this sentence exceeds 360 words → stop
        if current_words + sent_words > max_words:
            break

        final_sentences.append(sent)
        current_words += sent_words

    # Ensure final ends with punctuation
    paragraph = " ".join(final_sentences).strip()
    if not paragraph.endswith(('.', '!', '?')):
        # last sentence already ends correctly in TinyStories, but fallback safeguard:
        paragraph += "."

    return paragraph, idx

In [ ]:
# Generate file
output_filename = "output.txt"

with open(output_filename, "w", encoding="utf-8") as f:
    current_index = 0

    for _ in range(300):
        paragraph, current_index = build_paragraph(current_index)

        formatted = f'"""\n{paragraph}\n""",\n\n'
        f.write(formatted)

print("File created:", output_filename)

File created: output.txt
